In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 读取数据
train_data = pd.read_csv("mediaeval-2015-trainingset.txt", sep="\t")
test_data = pd.read_csv("mediaeval-2015-testset.txt", sep="\t")

# 提取文本和标签
train_texts = train_data["tweetText"].tolist()
train_labels = train_data["label"].apply(lambda x: 1 if x == "real" else 0).tolist()

test_texts = test_data["tweetText"].tolist()
test_labels = test_data["label"].apply(lambda x: 1 if x == "real" else 0).tolist()

# 划分验证集
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42
)
from transformers import BertTokenizer
from torch.utils.data import Dataset

# 指定本地路径
local_path = "F:\python\SVM_for_FakeNewsDetection-master\mBERT"

# 加载预训练的BERT分词器
tokenizer = BertTokenizer.from_pretrained(local_path)

class FakeNewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long),
        }

# 创建数据集实例
train_dataset = FakeNewsDataset(train_texts, train_labels, tokenizer, max_length=128)
val_dataset = FakeNewsDataset(val_texts, val_labels, tokenizer, max_length=128)
test_dataset = FakeNewsDataset(test_texts, test_labels, tokenizer, max_length=128)
from transformers import BertForSequenceClassification
import torch

# 加载预训练的BERT模型（2分类）
model = BertForSequenceClassification.from_pretrained(local_path, num_labels=2)

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm

# 定义DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 优化器
optimizer = AdamW(model.parameters(), lr=5e-5)

# 训练循环
epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        inputs = {
            "input_ids": batch["input_ids"].to(device),
            "attention_mask": batch["attention_mask"].to(device),
            "labels": batch["labels"].to(device),
        }
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predictions = torch.argmax(outputs.logits, dim=1)
        correct += (predictions == inputs["labels"]).sum().item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}, Accuracy: {correct / len(train_dataset):.4f}")
from sklearn.metrics import classification_report

# 评估函数
def evaluate(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            inputs = {
                "input_ids": batch["input_ids"].to(device),
                "attention_mask": batch["attention_mask"].to(device),
            }
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            labels = batch["labels"].cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels)
    return all_preds, all_labels

# 在验证集上评估
val_preds, val_labels = evaluate(model, val_loader)
print(classification_report(val_labels, val_preds, target_names=["Fake", "Real"]))

D:\Anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\Anaconda\envs\pytorch\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at F:\python\SVM_for_FakeNewsDetection-master\mBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
D:\Anaconda\envs\pytorch\lib\site

Epoch 1/10, Loss: 257.4466, Accuracy: 0.8462


100%|████████████████████████████████████████████████████████████████████████████████| 714/714 [03:13<00:00,  3.68it/s]


Epoch 2/10, Loss: 177.4807, Accuracy: 0.8997


100%|████████████████████████████████████████████████████████████████████████████████| 714/714 [03:16<00:00,  3.64it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████| 714/714 [03:12<00:00,  3.72it/s]


Epoch 9/10, Loss: 465.1454, Accuracy: 0.6544


100%|████████████████████████████████████████████████████████████████████████████████| 714/714 [03:15<00:00,  3.65it/s]


Epoch 10/10, Loss: 465.2360, Accuracy: 0.6551
              precision    recall  f1-score   support

        Fake       0.65      1.00      0.79      1854
        Real       0.00      0.00      0.00      1002

    accuracy                           0.65      2856
   macro avg       0.32      0.50      0.39      2856
weighted avg       0.42      0.65      0.51      2856



D:\Anaconda\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Anaconda\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Anaconda\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
